<a href="https://colab.research.google.com/github/jaideep11061982/GenAINotebooks/blob/main/LLama_3_1_8B_GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation

In [ ]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow

In [ ]:
# from huggingface_hub import login
# from google.colab import userdata

# hf_token = userdata.get('HF_TOKEN')
# login(hf_token)

### Unsloth

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [ ]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Load up `Llama 3.1 8B Instruct`, and set parameters

In [ ]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 512 # Can increase for longer reasoning traces
lora_rank = 16 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    # gpu_memory_utilization = 0.2, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

INFO 03-03 19:08:44 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/meta-llama-3.1-8b-instruct-unsloth-bnb-4bit with actual GPU utilization = 49.66%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 128.
Unsloth: vLLM's KV Cache can use up to 1.22 GB. Also swap space = 2 GB.
WARNING 03-03 19:08:51 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 03-03 19:09:11 config.py:549] This model

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

INFO 03-03 19:11:44 model_runner.py:1562] Graph capturing finished in 32 secs, took 0.52 GiB
INFO 03-03 19:11:44 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 45.89 seconds


tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [ ]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

QUESTION_PROMPT = """{}
A) {}
B) {}
C) {}
D) {}"""

def get_medQA_questions(split = "train") -> Dataset:
    data = load_dataset('GBaker/MedQA-USMLE-4-options')[split] # type: ignore
    data = data.map(lambda x: {
                                'prompt': [
                                    {'role': 'system', 'content': SYSTEM_PROMPT},
                                    {'role': 'user', 'content': QUESTION_PROMPT.format(x['question'],
                                                                                      x["options"]["A"],
                                                                                      x["options"]["B"],
                                                                                      x["options"]["C"],
                                                                                      x["options"]["D"])}
                                ],
                                'answer': x['answer_idx'] })
    return data.select(range(5000))

dataset_ = get_medQA_questions()
dataset = dataset_.remove_columns(["question","options","meta_info","answer_idx","metamap_phrases"])

README.md:   0%|          | 0.00/654 [00:00<?, ?B/s]

phrases_no_exclude_train.jsonl:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

phrases_no_exclude_test.jsonl:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1273 [00:00<?, ? examples/s]

Map:   0%|          | 0/10178 [00:00<?, ? examples/s]

In [ ]:
dataset[0]

{'answer': 'D',
 'prompt': [{'content': '\nRespond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n',
   'role': 'system'},
  {'content': 'A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?\nA) Ampicillin\nB) Ceftriaxone\nC) Doxycycline\nD) Nitrofurantoin',
   'role': 'user'}]}

In [ ]:
import re
from datasets import load_dataset, Dataset


XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def char_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r in ["A","B","C","D"] else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [ ]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = 512,
    # max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 6


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        char_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 6 | Gradient Accumulation steps = 1
\        /    Total batch size = 6 | Total steps = 250
 "-____-"     Number of trainable parameters = 41,943,040


-------------------- Question:
A 20-year-old man presents to the urgent care clinic complaining of nausea and vomiting for the past 2 hours. He just returned from a boating trip with his father, and while aboard they shared some packed potato salad and ham sandwiches. His dad denies any nausea or vomiting but does report minor dizziness. On examination he appears pale. The patient reports similar symptoms in the past when he was on a cruise trip to the Bahamas. What is the best medication for this patient at this time?
A) Diphenhydramine
B) Loperamide
C) Loratadine
D) Ondansetron 
Answer:
A 
Response:
<reasoning>
The patient's presentation of a 4-cm sharply defined liver mass with a hypoattenuated central scar on contrast-enhanced CT scan is highly suggestive of a benign condition known as an adenoma. The use of oral contraceptives increases the risk of developing liver adenomas, which are hormone-dependent and typically regression occurs with discontinuation of the contraceptive.

Con

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / char_reward_func,rewards / correctness_reward_func
1,0.000000,-0.089000,0.388354,199.166672,0.000011,-0.089000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.106500,0.141787,191.500000,0.000010,0.106500,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.125000,0.000000,64.000000,0.000008,0.125000,0.000000,0.000000,0.000000,0.000000
4,0.000000,-0.038833,0.343727,180.833344,0.000012,-0.038833,0.000000,0.000000,0.000000,0.000000
5,0.000000,-0.151833,0.417935,222.000000,0.000008,-0.151833,0.000000,0.000000,0.000000,0.000000
6,0.000000,-0.189500,0.356470,206.000000,0.000011,-0.189500,0.000000,0.000000,0.000000,0.000000
7,0.000000,-0.056667,0.386997,254.833344,0.000008,-0.056667,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.012333,0.165265,242.500000,0.000010,0.012333,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.008333,0.142783,121.833336,0.000015,0.008333,0.000000,0.000000,0.000000,0.000000
10,0.000000,-0.284000,0.412766,251.666672,0.000010,-0.284000,0.000000,0.000000,0.000000,0.000000


-------------------- Question:
A 28-year-old female suffering from a urinary tract infection is given trimethoprim-sulfamethoxazole (TMP-SMX) by her physician. Several days later, she begins to experience itchiness and joint pain. Laboratory and histologic analysis reveals vasculitis and antibody complexes deposited near the basement membrane of the glomerulus. What other serological finding is expected with this presentation?
A) Decreased levels of IgE
B) Increased levels of IgE
C) Increased serum levels of complement protein C3
D) Lowered serum levels of complement protein C3 
Answer:
D 
Response:
<reasoning>
The symptoms described (itchiness and joint pain) along with vasculitis and deposition of antibody complexes near the basement membrane of the glomerulus are indicative of an immune complex-mediated mechanism. Immune complex deposition leads to complement activation. The complement system plays a crucial role in the immune response and can activate the classical pathway in respo

IndexError: argmax(): Expected reduction dim 1 to have non-zero size.

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "How to treat acne?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:26<00:00, 26.76s/it, est. speed input: 1.53 toks/s, output: 17.64 toks/s]


'Treating acne involves a combination of self-care, lifestyle changes, and, if necessary, medical treatments. Here are some steps and tips to help you manage acne:\n\n**Self-Care:**\n\n1. **Keep your skin clean:** Wash your face twice a day with a gentle cleanser, making sure to remove all makeup and impurities.\n2. **Exfoliate regularly:** Use a gentle exfoliating scrub or a chemical exfoliant containing alpha-hydroxy acids (AHAs) or beta-hydroxy acids (BHAs) to remove dead skin cells and unclog pores.\n3. **Use non-comedogenic products:** Choose products labeled "non-comedogenic" or "oil-free" to avoid clogging your pores.\n4. **Avoid picking or squeezing:** Resist the temptation to squeeze or pick at pimples, as this can lead to further inflammation, scarring, and infection.\n5. **Get enough sleep:** Aim for 7-8 hours of sleep per night to help regulate hormones and reduce stress.\n\n**Lifestyle Changes:**\n\n1. **Maintain a healthy diet:** Eat a balanced diet rich in fruits, vegeta

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "How to treat acne?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output


Processed prompts: 100%|██████████| 1/1 [00:24<00:00, 24.96s/it, est. speed input: 2.52 toks/s, output: 18.03 toks/s]


'<Reasoning>\nTreating acne requires a combination of good skincare habits, lifestyle modifications, and sometimes medication. The goal is to reduce inflammation, prevent clogged pores, and promote healing. The approach to treating acne depends on the severity and type of acne, as well as the individual\'s skin type and sensitivity.\n\nHere are some general steps to treat acne:\n\n1. **Maintain good skincare routine**: Wash your face twice a day with a gentle cleanser to remove dirt, oil, and bacteria. Use a non-comedogenic (oil-free) cleanser that won\'t clog pores.\n2. **Exfoliate regularly**: Exfoliating helps remove dead skin cells and unclog pores. Use a gentle exfoliating scrub or a chemical exfoliant containing alpha-hydroxy acids (AHAs) or beta-hydroxy acids (BHAs) one to three times a week.\n3. **Use non-comedogenic products**: Choose products labeled "non-comedogenic" or "oil-free" to avoid clogging pores.\n4. **Avoid picking or popping pimples**: Resist the temptation to squ

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )